# 券商分配问题

### Author: Daniel 周亚楠
### Time: 2021/08/16

我们想通过调研券商研究员来了解一些上市公司的情况。热门的上市公司会有更多的券商和研究员cover，冷门的上市公司可能只有少数的券商和研究员cover。我们根据每家券商所cover的上市公司数量来判断它们综合实力的强弱，根据每个券商针对各个上市公司的研报数量来判断该券商对于各上市公司研究的深入程度。举例：下面数据表示东吴证券的综合实力（118家）没有中金（204家）强，但是东吴证券对于麦格米特这个公司的研究（22篇）比中金（16篇）更深入。

理想情况下，针对每个上市公司，我们希望找到研究最深入的券商，于此同时该券商的综合实力也是在cover该上市公司的券商中最强大的。但是实际情况中，我们并不能得到这个完美的解。另外，从差旅的角度出发，券商分布在不同城市，广泛调研势必带来成本负担（如：某冷门上市公司只有一家小券商cover，我们可能就倾向放弃该上市公司）。因此，在兼顾研究深度和券商综合实力的同时，我们希望通过调研尽可能少的券商，而cover尽可能多的上市公司。

通过建模给出一个满足上诉要求的调研方案，即：去拜访哪些券商，调研哪些上市公司。结果以excel形式输出，第一列为券商名，第二列为上市公司股票简称。

该project限时1周，将结果发送到zhuja@jsfund.cn

### Part 1: 尽量深入（研报数量尽可能多），不考虑券商分配是否平均

In [3]:
import numpy as np
import pandas as pd


class get_solution():

    def __init__(self):
        self.file = pd.read_excel("调研库.xlsx")
        self.qs_cover_low = 5
        self.qs_num = 15
        # self.select_file()


    def select_file(self):
        """
        如果'券商cover股票数'小于 qs_cover_low 则认为此券商没有意义删掉
        """
        file = self.file
        file = file[file['券商cover股票数'] > self.qs_cover_low]
        self.file = file

    def get_num(self):
        """
        将数据按照 '股票简称' 分类，去每个股票中有最大研报数量的券商
        建立哈希表，将每个股票对应需要调研的券商以 dictionary 的方式储存起来
        """
        file = self.file
        qs_num = self.qs_num

        file_deep = file.groupby(['股票简称']).agg({'研报数量':'max'}).reset_index()
        file_deep = pd.merge(file_deep, file.loc[:,['券商', '股票简称', '股票代码', '研报数量', '券商cover股票数']])
        file_deep = file_deep.drop_duplicates(subset=['股票简称'])

        # 建立哈希表，将每个股票对应需要调研的券商以 dictionary 的方式储存起来
        # 每个股票对应的券商是拥有最多研报的券商，即调研最深入的券商
        qs_stock_dict = file_deep.set_index('股票简称')['券商'].to_dict()

        # 按照券商分配股票的数量将不同的券商进行排序
        qs_stat = file_deep.groupby('券商').agg({'股票简称':'count'}).reset_index()
        qs_stat.columns = ['券商', '券商分配股票数量']
        qs_stat = qs_stat.sort_values(['券商分配股票数量'], ascending=False).reset_index().drop(['index'], axis=1)
        qs_stat = pd.merge(qs_stat, file_deep.loc[:, ['券商','券商cover股票数']].drop_duplicates(), on='券商', how='left')

        # 将目标中的需要拜访的券商分成两类，需要拜访的 'qs_stat_more' 和 不需要拜访的 'qs_stat_less'
        # 选择集中拜访 N 个券商， N = qs_num
        qs_stat_less = qs_stat.iloc[qs_num:, :]
        qs_stat_more = qs_stat.iloc[:qs_num, :]
        # qs_u 即 '券商 in use'，需要拜访的券商
        qs_u = list(qs_stat_more['券商'])
        self.qs_u = qs_u


        # 对每个不需要拜访的券商进行循环
        # 将其替换为需要拜访的券商
        for i in range(len(qs_stat_less)):
            qs = qs_stat_less.iloc[i, 0]  # 不需要拜访的券商

            # stock_target 改券商对应的股票
            stock_target = file_deep[file_deep['券商'] == qs]['股票简称']
            for stock in stock_target:
                qs_candidate = file[file['股票简称'] == stock]['券商']

                # 如果有该股票研报的券商在列表中，则选择替换为需要拜访的券商，break
                # 如果有该股票研报的券商都在不需要拜访的券商的列表中，标记为0，之后删除
                for num, qs_c in enumerate(qs_candidate):
                    if qs_c in qs_u:
                        qs_stock_dict[stock] = qs_c
                        # print(stock, qs_c)
                        break
                    if (num == len(qs_candidate)-1) and (qs_c not in qs_u):
                        qs_stock_dict[stock] = 0

        file_new = pd.DataFrame({'股票简称':qs_stock_dict.keys(),  '券商':qs_stock_dict.values()})
        file_new = file_new[file_new['券商'] != 0]
        qs_stat_new = file_new.groupby('券商').agg({'股票简称':'count'}).reset_index()
        qs_stat_new.columns = ['券商', '券商分配股票数量']
        qs_stat_new = qs_stat_new.sort_values(['券商分配股票数量'], ascending=False)
        qs_stat_new.index = list(range(len(qs_stat_new)))
        # qs_stat_new = pd.merge(qs_stat_new, file_deep.loc[:, ['券商','券商cover股票数']].drop_duplicates(), on='券商', how='left')

        self.qs_stock_dict = qs_stock_dict
        self.qs_stat = qs_stat_new

In [11]:
# 执行代码
gs = get_solution()
gs.get_num()

In [12]:
# 查看券商分配的统计结果
gs.qs_stat

,券商,券商分配股票数量
0,中金,61
1,中泰证券,58
2,申万宏源证券,54
3,中信证券,44
4,华泰证券,39
5,国泰君安,39
6,东吴证券,33
7,兴业证券,31
8,广发证券,29
9,国信证券,24


In [13]:
# 将 dictionary 以 DataFrame 的形式储存起来
dict = gs.qs_stock_dict
file = pd.DataFrame({'股票简称': dict.keys(), '券商': dict.values()})
file = pd.merge(file, gs.file.loc[:, [ '券商', '股票简称', '研报数量']], how='left')
file = file[file['券商'] != 0]
file = file.loc[:, ['券商', '股票简称', '研报数量']].sort_values('券商')

# 将第一个结果 outcome_part1.xlsx 以 excel 的形式输出
file.to_excel("outcome_part1.xlsx", index=False)

### Part 2: 考虑股票分配到券商尽可能平均

优化的函数使用了基尼系数(Gini index)或信息熵(information entropy)来衡量是否均衡  
优化目标为 mark = m * gini + n * total_num  
gini 是整个分配方法的基尼系数  
total_num 是整个分配方案的全部研报数量  
m，n 是两个系数，可以人工定义  

第一部分为 m * gini，用来控制券商分配到的股票是否均衡  
第二部分为 n * total_num，用来控制券商分配的

In [14]:
# 定义需要的函数

def get_gini(ser):
    """
    计算一个 Series 的基尼系数
    """
    proportion = ser/sum(ser)
    return 1 - sum(proportion * proportion)

def get_entropy(ser):
    """
    计算一个 Series 的信息熵
    """
    proportion = ser / sum(ser)
    return - sum(proportion * np.log2(proportion))

def get_mark(qs_stock_dict, file_org,  m, n):
    """
    计算分配方案的权重
    分配的公式为: mark = m * 基尼系数 + n * 研报总数量
    :param qs_stock_dict: 用于储存分配方案的哈希表
    :param file_org: 最原始的DataFrame，用于更新方案
    :param m: 基尼系数之前的系数
    :param n: 研报总数量之前的系数
    :return: file:分配方案的 DataFrame 形式
             qs_stat:分配方案的统计
             mark 本次分配方案的得分
    """
    file = pd.DataFrame({'股票简称': qs_stock_dict.keys(), '券商': qs_stock_dict.values()})
    file = pd.merge(file, file_org.loc[:, [ '券商', '股票简称', '研报数量']], how='left')
    file = file[file['券商'] != 0]
    qs_stat = file.groupby('券商').agg({'股票简称': 'count', '研报数量':'sum'}).reset_index()
    qs_stat.columns = ['券商', '券商分配股票数量', '研报数量']
    qs_stat = qs_stat.sort_values(['券商分配股票数量'], ascending=False).reset_index().drop(['index'], axis=1)
    gini = get_gini(qs_stat['券商分配股票数量'])
    total_num = sum(qs_stat['研报数量'])
    mark = m * gini + n * total_num
    # mark 应该是约高约好
    return file, qs_stat, mark


In [15]:
# 获取变量
qs_u = gs.qs_u
file_2 = gs.file.copy()
dict_2 = gs.qs_stock_dict.copy()
qs_stat_2 = gs.qs_stat.copy()

In [16]:
# 开始执行调整的循环
current_mark = 0     # 用 0 初始化本次方案的分数
epsilon = 1          # 设置 threshold，如果两次方案的差小于 epsilon，则停止更新

# 从分配到的股票最多的券商开始循环
# 计算分配方法分数的时候使用 m = 1000, n = 0，即不考虑研报数量的影响，要求分配方案尽可能平均
for i in range(len(file_2)):

    # qs 本次循环中需要拜访的券商
    qs = qs_stat_2.iloc[i, 0]
    stock_target = file_2[file_2['券商'] == qs]['股票简称']

    # 记录本次调整目标的股票
    for stock in stock_target:

        qs_candidate = file_2[file_2['股票简称'] == stock]['券商']
        for qs_c in qs_candidate:
            file, qs_stat, mark_o = get_mark(dict_2, file_2, 1000, 0)

            # 新选用的券商应该在需要拜访的券商列表 qs_u 中
            if qs_c in qs_u:
                # dict_2[stock]
                test_dict = dict_2.copy()
                test_dict[stock] = qs_c
                file, qs_stat, mark_t = get_mark(test_dict, file_2, 1000, 0)

                # 如果新方案的 mark 大于 旧方案的 mark，则更新方案
                # 一旦更新则停止本次的循环
                if mark_t > mark_o:
                    dict_2 = test_dict
                    print(i, stock, "旧分数：", mark_o, "新分数：", mark_t)
                    break

    # 如果两次循环中的
    print(qs_stat)
    if abs(current_mark - mark_o) < epsilon:
        break
    current_mark = mark_o

0 麦格米特 旧分数： 921.1317686511132 新分数： 921.318503475991
0 驰宏锌锗 旧分数： 921.318503475991 新分数： 921.481896447759
0 马钢股份 旧分数： 921.481896447759 新分数： 921.6063863310108
0 飞科电器 旧分数： 921.6063863310108 新分数： 921.839804862108
0 隧道股份 旧分数： 921.839804862108 新分数： 921.8942691860307
0 隆基股份 旧分数： 921.8942691860307 新分数： 921.9176110391404
0 陕西煤业 旧分数： 921.9176110391404 新分数： 922.0187590692825
0 阳光电源 旧分数： 922.0187590692825 新分数： 922.0421009223923
0 长盈精密 旧分数： 922.0421009223923 新分数： 922.1199070994246
0 长江证券 旧分数： 922.1199070994246 新分数： 922.3533256305219
0 锐科激光 旧分数： 922.3533256305219 新分数： 922.423351189851
0 金风科技 旧分数： 922.423351189851 新分数： 922.5711829262125
0 金隅集团 旧分数： 922.5711829262125 新分数： 922.7190146625741
0 金钼股份 旧分数： 922.7190146625741 新分数： 922.8512851635292
0 金螳螂 旧分数： 922.8512851635292 新分数： 922.9057494874518
0 金禾实业 旧分数： 922.9057494874518 新分数： 923.2091935778781
0 金地集团 旧分数： 923.2091935778781 新分数： 923.2714385195042
0 郑煤机 旧分数： 923.2714385195042 新分数： 923.4270508735688
0 邮储银行 旧分数： 923.4270508735688 新分数： 923.4892958151947
0 通

1 麦格米特 旧分数： 931.6978474921124 新分数： 931.7912149045513
1 首钢股份 旧分数： 931.7912149045513 新分数： 931.8223373753642
1 首旅酒店 旧分数： 931.8223373753642 新分数： 931.9001435523966
1 领益智造 旧分数： 931.9001435523966 新分数： 931.9079241700998
1 鞍钢股份 旧分数： 931.9079241700998 新分数： 931.9157047878031
1 隆基股份 旧分数： 931.9157047878031 新分数： 931.9390466409129
1 阳光电源 旧分数： 931.9390466409129 新分数： 932.0012915825388
1 长沙银行 旧分数： 932.0012915825388 新分数： 932.0557559064614
1 长春高新 旧分数： 932.0557559064614 新分数： 932.0868783772743
1 金风科技 旧分数： 932.0868783772743 新分数： 932.1024396126809
1 金域医学 旧分数： 932.1024396126809 新分数： 932.1180008480874
1 金地集团 旧分数： 932.1180008480874 新分数： 932.1646845543067
1 重庆银行 旧分数： 932.1646845543067 新分数： 932.2191488782295
1 通富微电 旧分数： 932.2191488782295 新分数： 932.2269294959326
1 通化东宝 旧分数： 932.2269294959326 新分数： 932.2891744375586
1 迪安诊断 旧分数： 932.2891744375586 新分数： 932.3514193791846
1 贝达药业 旧分数： 932.3514193791846 新分数： 932.4292255562169
1 豪迈科技 旧分数： 932.4292255562169 新分数： 932.4370061739202
1 蓝帆医疗 旧分数： 932.4370061739202 新分数： 932.4759092

In [17]:
# 查看券商分配的统计结果
qs_stat

,券商,券商分配股票数量,研报数量
0,东吴证券,34,367.0
1,中信证券,34,440.0
2,中泰证券,34,662.0
3,中金,34,378.0
4,兴业证券,34,442.0
5,华泰证券,34,315.0
6,国信证券,34,462.0
7,国泰君安,34,355.0
8,安信证券,34,424.0
9,广发证券,34,482.0


In [18]:
# 将 dictionary 以 DataFrame 的形式储存起来
dict = test_dict
file = pd.DataFrame({'股票简称': dict.keys(), '券商': dict.values()})
file = pd.merge(file, gs.file.loc[:, [ '券商', '股票简称', '研报数量']], how='left')
file = file[file['券商'] != 0]
file = file.loc[:, ['券商', '股票简称', '研报数量']].sort_values('券商')

# 将第二个结果 outcome_part2.xlsx 以 excel 的形式输出
file.to_excel("outcome_part2.xlsx", index=False)
